In [3]:
# CUDA: 11.3
# CUDNN: 8.6.0
# Python: 3.9.5

# Pytorch: 1.12.1

In [4]:
# pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113

## Import packages

In [5]:
import torch
import numpy as np
import time
from scipy.ndimage import distance_transform_edt as distance
from scipy.ndimage import _nd_image

c:\Users\wenbl13\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
device = torch.device("cuda:0")
print(device)

cuda:0


# Loss Functions

In [7]:
def dice_loss(score, target):
    target = target.float()
    smooth = 1e-5
    intersect = torch.sum(score * target)
    y_sum = torch.sum(target * target)
    z_sum = torch.sum(score * score)
    loss = (2 * intersect + smooth) / (z_sum + y_sum + smooth)
    loss = 1 - loss
    return loss

In [8]:
def hd_loss_2D(seg_soft, gt, seg_dtm, gt_dtm):
    """
    compute huasdorff distance loss for binary segmentation
    input: seg_soft: softmax results,  shape=(b,2,x,y,z)
           gt: ground truth, shape=(b,x,y,z)
           seg_dtm: segmentation distance transform map; shape=(b,2,x,y,z)
           gt_dtm: ground truth distance transform map; shape=(b,2,x,y,z)
    output: boundary_loss; sclar
    """

    delta_s = (seg_soft - gt) ** 2
    s_dtm = seg_dtm ** 2
    g_dtm = gt_dtm ** 2
    dtm = s_dtm + g_dtm
    multipled = torch.einsum('xy, xy->xy', delta_s, dtm)
    hd_loss = multipled.mean()

    return hd_loss

In [9]:
def hd_loss_3D(seg_soft, gt, seg_dtm, gt_dtm):
    """
    compute huasdorff distance loss for binary segmentation
    input: seg_soft: softmax results,  shape=(b,2,x,y,z)
           gt: ground truth, shape=(b,x,y,z)
           seg_dtm: segmentation distance transform map; shape=(b,2,x,y,z)
           gt_dtm: ground truth distance transform map; shape=(b,2,x,y,z)
    output: boundary_loss; sclar
    """

    delta_s = (seg_soft - gt) ** 2
    s_dtm = seg_dtm ** 2
    g_dtm = gt_dtm ** 2
    dtm = s_dtm + g_dtm
    multipled = torch.einsum('xyz, xyz->xyz', delta_s, dtm)
    hd_loss = multipled.mean()

    return hd_loss

# Benchmarks

In [10]:
num_range = range(1, 510, 10)

In [11]:
sizes_2D = []
sizes_3D = []

hd_min_2D = []
hd_std_2D = []
	
dice_min_2D = []
dice_std_2D = []
	
hd_min_3D = []
hd_std_3D = []
	
dice_min_3D = []
dice_std_3D = []

hd_3D_times = []
dice_3D_times = []

In [12]:
test_cases_2D = []

for n in num_range:
    _size = n**2
    sizes_2D.append(_size)
    test_cases_2D.append(torch.randint(0,2,(n,n)))

In [13]:
test_cases_3D = []

for n in num_range:
    _size = n**3
    sizes_3D.append(_size)
    test_cases_3D.append(torch.randint(0,2,(n,n,n)))

## 2D

In [55]:
#2D
hd_times = []
dice_times = []
for array in test_cases_2D:
  # HD GPU
  print("HD Size: ",  len(array))
  tfm1 = torch.from_numpy(distance(array)).cuda()
  n = len(array)
  g = torch.randint(0,2,(n,n))
  tfm2 = torch.from_numpy(distance(g)).cuda()
  g1 = g.cuda()
  array1 = array.cuda()
  a = []
  for j in range(1000): #Evaluations
    times1 = time.perf_counter_ns()
    hd = hd_loss_2D(array1, g1, tfm1, tfm2)
    times2 = time.perf_counter_ns()
    a.append(times2-times1)
    if sum(a) > (15*60*(10**9)):
      break
  hd_times.append(a)

for array in test_cases_2D:
  # Dice GPU
  print("Dice Size: ",  len(array))
  b = []
  n = len(array)
  g = torch.randint(0,2,(n,n)).cuda()
  array1 = array.cuda()
  for j in range(1000): #Evaluations
    times1 = time.perf_counter_ns()
    dice = dice_loss(array1, g)
    times2 = time.perf_counter_ns()
    b.append(times2-times1)
    if sum(b) > (20*60*(10**9)):
      break
  dice_times.append(b)
  



HD Size:  1
HD Size:  11
HD Size:  21
HD Size:  31
HD Size:  41
HD Size:  51
HD Size:  61
HD Size:  71
HD Size:  81
HD Size:  91
HD Size:  101
HD Size:  111
HD Size:  121
HD Size:  131
HD Size:  141
HD Size:  151
HD Size:  161
HD Size:  171
HD Size:  181
HD Size:  191
HD Size:  201
HD Size:  211
HD Size:  221
HD Size:  231
HD Size:  241
HD Size:  251
HD Size:  261
HD Size:  271
HD Size:  281
HD Size:  291
HD Size:  301
HD Size:  311
HD Size:  321
HD Size:  331
HD Size:  341
HD Size:  351
HD Size:  361
HD Size:  371
HD Size:  381
HD Size:  391
HD Size:  401
HD Size:  411
HD Size:  421
HD Size:  431
HD Size:  441
HD Size:  451
HD Size:  461
HD Size:  471
HD Size:  481
HD Size:  491
HD Size:  501
Dice Size:  1
Dice Size:  11
Dice Size:  21
Dice Size:  31
Dice Size:  41
Dice Size:  51
Dice Size:  61
Dice Size:  71
Dice Size:  81
Dice Size:  91
Dice Size:  101
Dice Size:  111
Dice Size:  121
Dice Size:  131
Dice Size:  141
Dice Size:  151
Dice Size:  161
Dice Size:  171
Dice Size:  181
Dice

In [15]:
# 3D
for array in test_cases_3D:
  # HD GPU
  print("HD Size: ", len(array))
  a = []
  tfm1 = torch.from_numpy(distance(array)).to(device)
  n = len(array)
  g = torch.randint(0,2,(n,n,n))
  tfm2 = torch.from_numpy(distance(g)).to(device)
  array1 = array.to(device)
  g1 = g.to(device)
  for j in range(1000): #Evaluations
    times1 = time.perf_counter_ns()
    hd = hd_loss_3D(array1, g1, tfm1, tfm2)
    times2 = time.perf_counter_ns()
    a.append(times2-times1)
    if sum(a) > (20*60*(10**9)):
      break
  hd_3D_times.append(a)

for array in test_cases_3D:
  # Dice GPU
  print("Dice Size: ",  len(array))
  b = []
  n = len(array)
  g = torch.randint(0,2,(n,n,n)).to(device)
  array1 = array.to(device)
  for j in range(1000): #Evaluations
    times1 = time.perf_counter_ns()
    dice = dice_loss(array1, g)
    times2 = time.perf_counter_ns()
    b.append(times2-times1)
    if sum(b) > (20*60*(10**9)):
      break
  dice_3D_times.append(b)

HD Size:  1
HD Size:  11
HD Size:  21
HD Size:  31
HD Size:  41
HD Size:  51
HD Size:  61
HD Size:  71
HD Size:  81
HD Size:  91
HD Size:  101
HD Size:  111
HD Size:  121
HD Size:  131
HD Size:  141
HD Size:  151
HD Size:  161
HD Size:  171
HD Size:  181
HD Size:  191
HD Size:  201
HD Size:  211
HD Size:  221
HD Size:  231
HD Size:  241
HD Size:  251
HD Size:  261
HD Size:  271
HD Size:  281
HD Size:  291
HD Size:  301
HD Size:  311
HD Size:  321
HD Size:  331
HD Size:  341
HD Size:  351
HD Size:  361
HD Size:  371
HD Size:  381
HD Size:  391
HD Size:  401
HD Size:  411
HD Size:  421
HD Size:  431
HD Size:  441
HD Size:  451
HD Size:  461
HD Size:  471
HD Size:  481
HD Size:  491
HD Size:  501
Dice Size:  1
Dice Size:  11
Dice Size:  21
Dice Size:  31
Dice Size:  41
Dice Size:  51
Dice Size:  61
Dice Size:  71
Dice Size:  81
Dice Size:  91
Dice Size:  101
Dice Size:  111
Dice Size:  121
Dice Size:  131
Dice Size:  141
Dice Size:  151
Dice Size:  161
Dice Size:  171
Dice Size:  181
Dice

In [16]:
for i in hd_times:
  hd_min_2D.append(torch.min(torch.FloatTensor(i)).numpy().tolist())
  hd_std_2D.append(torch.std(torch.FloatTensor(i), unbiased = False).numpy().tolist())

In [17]:
for i in dice_times:
  dice_min_2D.append(torch.min(torch.FloatTensor(i)).numpy().tolist())
  dice_std_2D.append(torch.std(torch.FloatTensor(i), unbiased = False).numpy().tolist())

In [18]:
for i in hd_3D_times:
  hd_min_3D.append(torch.min(torch.FloatTensor(i)).numpy().tolist())
  hd_std_3D.append(torch.std(torch.FloatTensor(i), unbiased = False).numpy().tolist())

In [19]:
for i in dice_3D_times:
  dice_min_3D.append(torch.min(torch.FloatTensor(i)).numpy().tolist())
  dice_std_3D.append(torch.std(torch.FloatTensor(i), unbiased = False).numpy().tolist())

### Save

In [20]:
import pandas as pd

In [21]:
hd_min = np.array(hd_min_2D)
dice_min = np.array(dice_min_2D)
hd_3D_min = np.array(hd_min_3D)
dice_3D_min = np.array(dice_min_3D)
hd_std = np.array(hd_std_2D)
dice_std = np.array(dice_std_2D)
hd_3D_std = np.array(hd_std_3D)
dice_3D_std = np.array(dice_std_3D)

In [22]:
data2D = {'sizes_2D': sizes_2D, 'hd_min_2D_purePython_GPU': hd_min, 'dice_min_2D_purePython_GPU': dice_min, 'hd_std_2D_purePython_GPU': hd_std, 'dice_std_2D_purePython_GPU': dice_std}
data3D = {'sizes_3D': sizes_3D, 'hd_min_3D_purePython_GPU': hd_3D_min, 'dice_min_3D_purePython_GPU': dice_3D_min, 'hd_std_3D_purePython_GPU': hd_3D_std, 'dice_std_3D_purePython_GPU':dice_3D_std}

In [23]:
dataframe2D = pd.DataFrame(data2D)
dataframe3D = pd.DataFrame(data3D)

In [24]:
dataframe2D.to_csv("C:/Users/wenbl13/Desktop/Ashwin-Timing/distance-transforms/purePython_Loss_2D_jan4_GPU.csv")
dataframe3D.to_csv("C:/Users/wenbl13/Desktop/Ashwin-Timing/distance-transforms/purePython_Loss_3D_jan4_GPU.csv")

In [25]:
dataframe2D

,sizes_2D,hd_min_2D_purePython_GPU,dice_min_2D_purePython_GPU,hd_std_2D_purePython_GPU,dice_std_2D_purePython_GPU
0,1,49400.0,79000.0,2.524915e+07,15924.915039
1,121,49500.0,80700.0,1.366043e+04,35737.605469
2,441,49500.0,84300.0,9.174281e+03,18744.773438
3,961,51100.0,84100.0,1.163829e+04,31827.390625
4,1681,49400.0,79300.0,3.376206e+04,20732.798828
5,2601,49500.0,81000.0,1.666096e+04,14775.595703
6,3721,49800.0,81000.0,1.464287e+04,43750.152344
7,5041,49500.0,81100.0,1.101053e+04,19643.193359
8,6561,49900.0,81500.0,3.503232e+04,13292.008789
9,8281,49200.0,84200.0,1.723897e+04,41286.238281


In [26]:
x = [i for i in range(1, 1000, 100)]

In [27]:
import matplotlib.pyplot as plt


In [28]:
# plt.figure(figsize=(13, 13))
# plt.plot(x, dataframe['hd_min_2D_purePython'], label = 'hd_min_2D')
# plt.plot(x, dataframe['dice_min_2D_purePython'], label = 'dice_min_2D')
# plt.plot(x, dataframe['hd_min_3D_purePython'], label = 'hd_min_3D')
# plt.plot(x, dataframe['dice_min_3D_purePython'], label = 'dice_min_3D')
# plt.xlabel('Array_Size')
# plt.ylabel('Time (seconds)')
# plt.legend()
# plt.show()

In [29]:
# y = torch.FloatTensor([[0, 1, 0],[1,1,1], [1,0,0]])
# x = torch.FloatTensor([[0, 0, 0], [1, 0, 1], [1, 1, 0]])
# y_dtm = torch.FloatTensor([[2, 65, 78], [4, 7, 3], [3, 6, 2]])
# x_dtm = torch.FloatTensor([[34, 31, 6], [21, 45, 78], [15, 456, 3]])

# # rslt_cpu = hd_loss_2D(x, y, x_dtm, y_dtm)

# y_GPU = y.cuda()
# x_GPU  = x.cuda()
# y_dtm_GPU  = y_dtm.cuda()
# x_dtm_GPU  = x_dtm.cuda()


# rslt_gpu = hd_loss_2D(x_GPU, y_GPU, x_dtm_GPU, y_dtm_GPU)

# # print("cpu:", rslt_cpu)
# print("gpu:", rslt_gpu)

In [30]:
y.size()

NameError: name 'y' is not defined